<a href="https://colab.research.google.com/github/gshartnett/introAI/blob/main/homeworks/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Modern AI - HW 4
**Student's Name**: YOUR NAME HERE  
Instructor: Gavin Hartnett  
PRGS, Winter Quarter 2022  

This HW is worth 12.5% of your grade. Complete the assignment by making a local copy of this Colab Notebook and filling in the responses in your local copy. If you do not know how to typset math in LaTeX, feel free to email me a scanned piece of paper with your work instead. 

In [ ]:
## imports

## numpy
import numpy as np
from numpy.random import default_rng
rng = default_rng(123)

## other useful data analysis libraries
import pandas as pd
from sklearn import neighbors

## plotting
from matplotlib.colors import ListedColormap
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [ ]:
## some commands to make the plots look nicer
plt.style.use('seaborn-white')
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
## set the device variable
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Problem 1: PCA

**Part A)**  
Following my example in the Week 4 Notebook, download the *training part* of the [Fashion MNIST](https://pytorch.org/vision/stable/datasets.html#fashion-mnist) dataset. (This time, without any transformations applied). 
- What are the dimensions of the images?
- Make a 10x10 plot of 100 random images to get a sense for what the dataset looks like. 

In [ ]:
## your code here

**Part B)**  
Using any approach you find convenient, organize the data into one large numpy array $X$. Flatten the pixels so that $X$ is a matrix with dimensions $N \times p$, where $N$ is the number of images in the training set and $p=n^2$ is the square of the pixel size. Also collect all the labels in a $N$-dimensional vector $y$. 

In [ ]:
## your code here

**Part C)**  
Apply the [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) implementation of PCA to the data array $X$ from the previous part. Project the data down to the first 2 principal components. The result should be a $N \times 2$ dimensional array `X_reduced`. 


In [ ]:
## your code here

60,000 data points is going to be a bit unwieldy for what we're going to do next, so let's just grab the first 5000 datapoints.

In [ ]:
X_reduced = X_reduced[0:5000, :]
y = y[0:5000]

## Problem 2: K-Means Clustering

In this problem we will implement the K-means clustering algorithm and apply it to the PCA-reduced Fashion MNIST dataset. Recall that K-means is an unsupervised algorithm designed to find the best clustering assignments based on minimizing the squared Euclidean distance between data points. By setting $K=10$, we will investigate to what extent these clusters correspond to the labels within the Fashion MNIST dataset. 

**Part A)**
Implement *from scratch* K-means clustering with $K=10$. The pseudocode version of the algorithm is:
- Initialize the cluster assignments $c(i)$ for $i=1,...,N$
- Initialize the means or centroids, $\boldsymbol{\mu}_k$ for $k=1,...,K$. 
- Until convergence, do:
    - for each $i=1,...,N$, update  
$$c(i) = \text{argmin}_k || \boldsymbol{x}_i - \boldsymbol{\mu}_k||^2$$
    - for each $k=1,...,K$, update  
$$\boldsymbol{\mu}_k = \frac{1}{N_k} \sum_{i: c(i)=k} \boldsymbol{x}_i$$

This part will be worth the same number of points as 3 ordinary parts (6 points total).

In [ ]:
## your code here
## Note: the final output should be a list of label assignments label_preds for each i=1,...,N

Now let's plot the result. The below code should run if `X_reduced` and `label_preds` are in the proper format. 

In [ ]:
# Step size of the mesh. Decrease to increase the quality of the VQ.
h = 0.02  # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = X_reduced[:, 0].min() - 1, X_reduced[:, 0].max() + 1
y_min, y_max = X_reduced[:, 1].min() - 1, X_reduced[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = model.predict_labels(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the K-means decision boundaries
fig = plt.figure(figsize=(10,10))
plt.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    cmap=plt.cm.Paired,
    aspect="auto",
    origin="lower",
)

## Plot the raw data
plt.plot(X_reduced[:, 0], X_reduced[:, 1], "k.", markersize=2)

## Plot the centroids as a white X
centroids = model.centroids
plt.scatter(
    centroids[:, 0],
    centroids[:, 1],
    marker="x",
    s=200,
    color="w",
    zorder=10,
    linewidth=1
)

## Make the plot pretty
plt.title(
    "K-means clustering on PCA-reduced Fashion MNIST\n"
    "Centroids are marked with white cross"
)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
#plt.xticks(())
#plt.yticks(())
plt.show()

This is perhaps an unusual application of K-means as in this case we already know a good labeling/clustering scheme, namely the labels associated with the original dataset. The code block below maps our 10 K-means labels to the true 10 labels and then checks our accuracy. 

In [ ]:
label_matrix = np.zeros((10, 10))
for i in range(X_reduced.shape[0]):
    label_matrix[y[i], label_preds[i]] += 1

argmax = np.argmax(label_matrix, axis=1)
my_dic = {i:argmax[i] for i in range(len(argmax))}

y_mapped = np.vectorize(my_dic.get)(y)
print('accuracy of K-means: %.2f' %np.mean(y_mapped == label_preds))

**Part B)**  
For fun (and so you know the easier way to do this in the future), use [scikit-learn's implementation of K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), and then make the same plot as above.

In [ ]:
## your code here

## Problem 3: Policy2Vec

In this problem you will gain experience applying [Gensim's implementation of *doc2vec*](https://radimrehurek.com/gensim/models/doc2vec.html) to policy-relevant documents. This problem is inspired by [some work I did a few years ago](https://github.com/RANDCorporation/policy2vec).

**Part A)**  
For this problem we will look at Executive Orders, which can be obtained [here](https://www.federalregister.gov/presidential-documents/executive-orders). This website will allow you to download a .csv file containing the urls of individual Executive Order pdf files. I went through the trouble of downloading all these and converting them to raw text form. You can find this data in the Teams folder. Download the data to your local machine, and upload it using the code below. 

Note: I'm being easy on you - you don't need to write any code, just follow the instructions and upload the data correctly.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
## the files are uploaded into a dict, put them into a list of filenames and texts
filenames = list(uploaded.keys())
texts = list(uploaded.values())

In [ ]:
texts[0][0:10]

The text was uploaded as a bytes object (hence the b'xxxx' format). Let's convert this to a string format.

In [ ]:
for i in range(len(texts)):
    texts[i] = texts[i].decode('utf-8')

In [ ]:
texts[0][0:10]

In [ ]:
## let's inspect the text
print(texts[0][2000:3000])

**Part B)**  
Preprocess the text. I'm going to leave this rather open-ended to encourage you to play with it, since as I mentioned in class, preprocessing often ends up being the task you spend the most time on. Also, differences in how the data is preprocessed can often be more influential than the choice of algorithm. At the bare minimum, the text needs to be tokenized, i.e., 

`data = [data[0], data[1], data[2], ..., data[n]]`

where 

`data[i] = [token 0 of document i, token 1 of document i, ... ]`

and where n = 1, ..., number of executive order documents. As an example, if the texts were "see spot run", "he is fast", then we want something like

`data = [['see', 'spot', 'run'], ['he', 'is', 'fast']]`.

Additional things to explore are:
- removing stop words
- lemmatizing
- removing whitespace, special characters
- changing to lower-case



In [ ]:
## your code here

**Part C)**  
Train a Doc2Vec model using Gensim. This can be done with one line of code, see this [docs page](https://radimrehurek.com/gensim/models/doc2vec.html
) for help.

In [ ]:
## first we need to format the data in the way Gensim expects
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data)]

## your code here

**Part D)**  
Define a function that implements the cosine similarity $S_C(\boldsymbol{v}_1, \boldsymbol{v}_2)$, for two vectors $\boldsymbol{v}_1, \boldsymbol{v}_2$.

In [ ]:
## your code here

**Part E)**  
Use the `model.infer_vector()` method to find the EO document whose vector is most similar (in terms of cosine similarity) to:

"The US is considering sanctioning Russian oligarchs in response to the invasion of Crimea." 

This will require you to compute $S_C(v(d), v_*)$ for all $d \in D$, where $v_*$ is the vector for the above prompt, and $v(d)$ is the vector for document $d$. Then you'll have to find the $d$ for which this similarity is largest.  

In [ ]:
prompt = 'The US is considering sanctioning Russian oligarchs connected with Vladimir Putin in response to the invasion of Crimea in Ukraine.'

In [ ]:
## your code here

In [ ]:
print(texts[imax])